In [1]:
import numpy as np
import pandas as pd
import spacy

from autocorrect import Speller
spell = Speller()

import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

from tqdm import tqdm_notebook as tqdm
from pprint import pprint


In [3]:
csv = pd.read_csv('metadata.csv')
csv.head(1)

,Title,Author 1,Affiliation A1,Country A1,Gender A1,Author 2,Affiliation A2,Country A2,Gender A2,Author 3,...,Affiliation A7,Country A7,Gender A7,Abstract,Year,Type,Conference/workshop/... title,Volume,Special issue,Blog extract
0,Believing in belonging': Researching belief wi...,Abby Day,University of Sussex,England,F,.,.,.,.,.,...,.,.,.,A unique research method was designed to answe...,2009,Conference,Non-religion and Secularity: New Empirical Per...,.,.,.


In [4]:
# nlp = spacy.load("en_core_web_sm")
nlp = spacy.load("en_core_web_lg")

stopList_byType = ['ADV', 'PUNCT' , 'ADP', 'DET', 'NUM', 'SYM', 'SPACE', 'SCONJ', 'AUX', 'PART', 'CCONJ', 'PRON', 'ADJ', 'VERB']

doc_list = []

for index, row in csv.iterrows():

    string = row['Title'].lower() + '. ' + row['Abstract'].lower()
    doc = nlp(string)

    lemmas = ''
    
    for i, token in enumerate(doc):
        
        if not token.pos_ in stopList_byType and not token.is_stop and len(token.lemma_) > 2 and len(token.lemma_) < 16:

            if len(lemmas) == 0:
                lemmas = token.lemma_
            else:
                lemmas += ',' + token.lemma_

    csv.loc[index, 'lemmas'] = lemmas
    doc_list.append(lemmas.split(','))

csv.to_csv('metadata.csv', index=False)

csv.head()

,Title,Author 1,Affiliation A1,Country A1,Gender A1,Author 2,Affiliation A2,Country A2,Gender A2,Author 3,...,Country A7,Gender A7,Abstract,Year,Type,Conference/workshop/... title,Volume,Special issue,Blog extract,lemmas
0,Believing in belonging': Researching belief wi...,Abby Day,University of Sussex,England,F,.,.,.,.,.,...,.,.,A unique research method was designed to answe...,2009,Conference,Non-religion and Secularity: New Empirical Per...,.,.,.,"belief,question,research,method,question,peopl..."
1,Members of Atheist/Freethought/Secularist orga...,Björn Mastiaux,University of Düsseldorf,Germany,M,.,.,.,.,.,...,.,.,I would like to contribute a paper documenting...,2009,Conference,Non-religion and Secularity: New Empirical Per...,.,.,.,"member,freethought,organization,germany,state,..."
2,.,John Nash,Oxford Brookes University,England,M,.,.,.,.,.,...,.,.,.,2009,Conference,Non-religion and Secularity: New Empirical Per...,.,.,.,
3,A secular mind,Jonathan A. Lanman,University of Oxford,England,M,.,.,.,.,.,...,.,.,Cognitive anthropologists argue that we can be...,2009,Conference,Non-religion and Secularity: New Empirical Per...,.,.,.,"mind,anthropologist,question,insight,science,b..."
4,Theoretical and methodological resources for b...,Kim Knott,University of Leeds,England,F,.,.,.,.,.,...,.,.,"The secular condition, secularism and the asso...",2009,Conference,Non-religion and Secularity: New Empirical Per...,.,.,.,"resource,boundary,religion,religion,condition,..."
